Project1_ IMDb Top 250 Movies Reviews

In [23]:
import requests
from bs4 import BeautifulSoup

# URL of IMDb Top 250 Movies
url = "https://www.imdb.com/chart/top/?sort=rank%2Casc"

# User-Agent for Microsoft Edge
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0 Edg/91.0.864.48"
}

# Send a GET request with the headers
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.text, 'html.parser')

    print("HTML content fetched and saved successfully.")
else:
    print(f"Failed to fetch page. Status code: {response.status_code}")


HTML content fetched and saved successfully.


In [25]:

# Parse the HTML content of the page using Beautiful Soup
soup = BeautifulSoup(response.content, 'html.parser')

In [27]:
response

<Response [200]>

In [ ]:
Extract Movie Data

In [29]:
# Create empty lists to store movie titles, years, ratings, and URLs
titles = []
years = []
ratings = []

In [31]:
# Find all the movie items in the HTML
movie_items = soup.find_all('item')
movie_items

[]

In [33]:
# Loop through the rows to extract movie information
for row in movie_items:
    # Extract title
    title_column = row.find('td', class_='titleColumn')
    if title_column:  # Check if title_column is not None
        title = title_column.a.text  # Get the movie title
        titles.append(title)  # Append title to the list
       # movie_items = soup.find_all('tr')  # or whatever the tag/selector is

        # Extract year
        year = title_column.span.text.strip('()')  # Remove parentheses
        years.append(year)  # Append year to the list

        # Extract rating
        rating_column = row.find('td', class_='ratingColumn imdbRating')
        if rating_column:
            rating = rating_column.strong.text  # Get the rating
            ratings.append(rating)  # Append rating to the list

In [35]:
print(movie_items)

[]


In [37]:
# Display the first few movie titles, years, and ratings
print("Titles:", titles[:5])
print("Years:", years[:5])
print("Ratings:", ratings[:5])

Titles: []
Years: []
Ratings: []


In [39]:
# Find the JSON data in the script tag
import json

json_data = soup.find('script', type='application/ld+json')

In [41]:
if json_data:
    # Parse the JSON data
    data = json.loads(json_data.string)

In [43]:
# Extract titles and ratings
titles = []
urls = []
descriptions = []
best_ratings = []
worst_ratings = []
ratings = []
genres = []
durations = []

In [45]:
# Check if the data contains the expected structure
if 'itemListElement' in data:
    for item in data['itemListElement']:
        movie = item['item']
        
        # Extract movie details
        titles.append(movie['name'])  # Movie name
        urls.append(movie['url'])  # Movie URL
        descriptions.append(movie['description'])  # Movie description
        
        # Extract ratings (best, worst, actual rating)
        best_ratings.append(movie['aggregateRating']['bestRating'])  # Best rating
        worst_ratings.append(movie['aggregateRating']['worstRating'])  # Worst rating
        ratings.append(float(movie['aggregateRating']['ratingValue']))  # Rating value
        
        genres.append(movie['genre'])  # Movie genre
        durations.append(movie['duration'])  # Movie duration

In [47]:
import pandas as pd

# Create a DataFrame to store the scraped data
df = pd.DataFrame({
    'Title': titles,
    'URL': urls,
    'Description': descriptions,
    'Best Rating': best_ratings,
    'Worst Rating': worst_ratings,
    'Rating': ratings,
    'Genre': genres,
    'Duration': durations
})

# Print the first few rows of the DataFrame
df.head()

,Title,URL,Description,Best Rating,Worst Rating,Rating,Genre,Duration
0,The Shawshank Redemption,https://www.imdb.com/title/tt0111161/,A banker convicted of uxoricide forms a friend...,10,1,9.3,Drama,PT2H22M
1,The Godfather,https://www.imdb.com/title/tt0068646/,The aging patriarch of an organized crime dyna...,10,1,9.2,"Crime, Drama",PT2H55M
2,The Dark Knight,https://www.imdb.com/title/tt0468569/,When a menace known as the Joker wreaks havoc ...,10,1,9.0,"Action, Crime, Drama",PT2H32M
3,The Godfather Part II,https://www.imdb.com/title/tt0071562/,The early life and career of Vito Corleone in ...,10,1,9.0,"Crime, Drama",PT3H22M
4,12 Angry Men,https://www.imdb.com/title/tt0050083/,The jury in a New York City murder trial is fr...,10,1,9.0,"Crime, Drama",PT1H36M


In [49]:
# Save the DataFrame to a CSV file
df.to_csv('topIMBDmovies.csv', index=False) 
print("Data saved in CSV format'")

Data saved in CSV format'
